In [ ]:
import json, os
import pandas as pd
import numpy as np
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.visualize import Visualizer, _normalize_world_keypoints
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.akpd_scorer import AKPDNormalizationTransform, AKPDScorerNetwork
from aquabyte.akpd import AKPD
from matplotlib import pyplot as plt
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from collections import defaultdict

pd.set_option('display.max_rows', 500)

<h1> Load base dataset </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from keypoint_annotations
    where keypoints is not null
    and keypoints -> 'leftCrop' is not null
    and keypoints -> 'rightCrop' is not null
    limit 100;
"""
good_df = rds_access_utils.extract_from_database(query)

In [ ]:
query = """
    select * from keypoint_annotations
    where is_partial = TRUE
    limit 100;
"""
bad_df = rds_access_utils.extract_from_database(query)

<h1> Run Inference </h1>

In [ ]:
aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))
akpd_scorer_network = torch.load('/root/data/alok/biomass_estimation/playground/akpd_scorer_model.pb')
akpd = AKPD(aws_credentials)

normalize_centered_2D_transform_akpd = NormalizeCentered2D(rotate=False, center=True)
akpd_normalization_transform = AKPDNormalizationTransform()
to_tensor_transform = ToTensor()


In [ ]:
analysis_df = pd.DataFrame()
for idx, row in good_df.iterrows():
    row_to_append = {}
    keypoints = row.keypoints
    left_keypoints, right_keypoints = keypoints['leftCrop'], keypoints['rightCrop']
    left_keypoints_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in left_keypoints}
    right_keypoints_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in right_keypoints}
    
    # append disparity values
    for bp in BODY_PARTS:
        row_to_append['disp_{}'.format(bp)] = left_keypoints_dict[bp][1] - right_keypoints_dict[bp][1]
    
    # run AKPD scoring network
    input_sample = {
        'keypoints': keypoints,
        'cm': row.camera_metadata,
        'stereo_pair_id': idx,
        'single_point_inference': True
    }
    
    # append AKPD score
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_akpd.__call__(input_sample)

    akpd_normalized_kps = akpd_normalization_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(akpd_normalized_kps)
    score = akpd_scorer_network(tensorized_kps['kp_input']).item()
    row_to_append['score'] = score
    row_to_append['is_good'] = True
    analysis_df = analysis_df.append(row_to_append, ignore_index=True)



In [ ]:
for idx, row in bad_df.iterrows():
    row_to_append = {}
    left_crop_url, right_crop_url = row.left_image_url, row.right_image_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata
    akpd_keypoints = akpd.predict_keypoints(left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata)[0]
    left_keypoints, right_keypoints = akpd_keypoints['leftCrop'], akpd_keypoints['rightCrop']
    left_keypoints_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in left_keypoints}
    right_keypoints_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in right_keypoints}
    for bp in BODY_PARTS:
        row_to_append['disp_{}'.format(bp)] = left_keypoints_dict[bp][1] - right_keypoints_dict[bp][1]
        
    # run AKPD scoring network
    input_sample = {
        'keypoints': akpd_keypoints,
        'cm': row.camera_metadata,
        'stereo_pair_id': idx,
        'single_point_inference': True
    }
    
    # append AKPD score
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_akpd.__call__(input_sample)

    akpd_normalized_kps = akpd_normalization_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(akpd_normalized_kps)
    score = akpd_scorer_network(tensorized_kps['kp_input']).item()
    row_to_append['score'] = score
    row_to_append['is_good'] = False
    analysis_df = analysis_df.append(row_to_append, ignore_index=True)
    

<h1> Compute Original Precision / Recall </h1>

In [ ]:
threshold = 0.5
good_mask = analysis_df.is_good == True
predicted_good_mask = analysis_df.score > threshold
recall = analysis_df[good_mask & predicted_good_mask].shape[0] / analysis_df[good_mask].shape[0]
precision = analysis_df[good_mask & predicted_good_mask].shape[0] / analysis_df[predicted_good_mask].shape[0]
print('Original Precision: {}'.format(precision))
print('Original Recall: {}'.format(recall))

<h1> Compute New Precision / Recall </h1>

In [ ]:
analysis_df[~good_mask]

In [ ]:
np.percentile(good_max_y_deviations, 99.5)

In [ ]:
for bp in BODY_PARTS:
    print('{}: {}'.format(bp, np.percentile(np.abs(y_deviation_values[bp]), 50)))

In [ ]:
def display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints, side='both', overlay_keypoints=True, show_labels=False):
    assert side == 'left' or side == 'right' or side == 'both', \
        'Invalid side value: {}'.format(side)

    if side == 'left' or side == 'right':
        fig, ax = plt.subplots(figsize=(20, 10))
        image_f = left_image_f if side == 'left' else right_image_f
        keypoints = left_keypoints if side == 'left' else right_keypoints
        image = plt.imread(image_f)
        ax.imshow(image)

        if overlay_keypoints:
            for bp, kp in keypoints.items():
                ax.scatter([kp[0]], [kp[1]], color='red', s=1)
                if show_labels:
                    ax.annotate(bp, (kp[0], kp[1]), color='red')
    else:
        fig, axes = plt.subplots(2, 1, figsize=(20, 20))
        left_image = plt.imread(left_image_f)
        right_image = plt.imread(right_image_f)
        axes[0].imshow(left_image)
        axes[1].imshow(right_image)
        if overlay_keypoints:
            for bp, kp in left_keypoints.items():
                axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
                if show_labels:
                    axes[0].annotate(bp, (kp[0], kp[1]), color='red')
            for bp, kp in right_keypoints.items():
                axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
                if show_labels:
                    axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
tdf = bad_df.copy(deep=True)
idx = 21
left_crop_url, right_crop_url = tdf.left_image_url.iloc[idx], tdf.right_image_url.iloc[idx]
left_crop_metadata, right_crop_metadata = tdf.left_crop_metadata.iloc[idx], tdf.right_crop_metadata.iloc[idx]
left_image_f, _, _ = s3_access_utils.download_from_url(left_crop_url)
right_image_f, _, _= s3_access_utils.download_from_url(right_crop_url)
akpd_keypoints = akpd.predict_keypoints(left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata)[0]
left_keypoints_list, right_keypoints_list = akpd_keypoints['leftCrop'], akpd_keypoints['rightCrop']
left_keypoints = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in left_keypoints_list}
right_keypoints = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in right_keypoints_list}
display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints, show_labels=True)

In [ ]:
# run AKPD scoring network
input_sample = {
#     'keypoints': akpd_keypoints[0],
    'keypoints': akpd_keypoints,
    'cm': tdf.camera_metadata.iloc[idx],
    'stereo_pair_id': idx,
    'single_point_inference': True
}
nomralized_centered_2D_kps = \
    normalize_centered_2D_transform_akpd.__call__(input_sample)

akpd_normalized_kps = akpd_normalization_transform.__call__(nomralized_centered_2D_kps)
tensorized_kps = to_tensor_transform.__call__(akpd_normalized_kps)
score = akpd_scorer_network(tensorized_kps['kp_input']).item()
print(score)

In [ ]:
for i, X_batch in enumerate(bad_dataloader_train):
    if i == 2:
        print(X_batch)
        break

In [ ]:
%matplotlib inline
plt.gca().set_aspect('equal', adjustable='box')
X = X_batch['kp_input'].numpy().squeeze()
plt.scatter(X[:, 0], X[:, 1])
plt.scatter(X[:, 2], X[:, 3])
plt.grid()
plt.show()